In [ ]:
# Run this code to analyse the whole database of videos and process it

### 1)Imports and setup

In [1]:
import os
import base64
import aiohttp
import asyncio
import json
import imageio
import re
import time
from PIL import Image
import numpy as np
import colorsys
import aiofiles
import nest_asyncio
from tqdm.asyncio import tqdm
from dotenv import load_dotenv
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
import logging
from datetime import datetime
import csv
from rapidfuzz import process, fuzz


# Configure logging
logging.basicConfig(level=logging.INFO)

# Load OpenAI API key from .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Concurrency limit
semaphore = asyncio.Semaphore(5)

# A dictionary to store characters across frames
character_frames = {}


In [2]:
#TRACK API USAGE CALLS

# Initialize API usage tracking
api_usage = {
    "total_api_calls": 0,
    "total_tokens_used": 0,
    "model_used": "gpt-4"  # Assuming you're using GPT-4
}


### 2) Video Analysis Functions

In [3]:
def analyze_video(video_path, threshold=27.0):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"The video file {video_path} does not exist.")
    
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()
    scene_manager.add_detector(ContentDetector(threshold=threshold))

    video_manager.set_downscale_factor()
    video_manager.start()

    scene_manager.detect_scenes(frame_source=video_manager)
    scene_list = scene_manager.get_scene_list()

    video_manager.release()

    logging.info(f'Detected {len(scene_list)} scenes:')
    for i, scene in enumerate(scene_list):
        logging.info(f'Scene {i + 1}: Start {scene[0].get_timecode()} / Frame {scene[0].get_frames()}, '
              f'End {scene[1].get_timecode()} / Frame {scene[1].get_frames()}')

    return scene_list

def get_video_length(video_path):
    # You can use a tool like OpenCV, ffmpeg, or similar to calculate video length
    import cv2
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_length = frame_count / fps
    cap.release()
    return video_length

### 3) Frame Extraction Function

In [4]:
def extract_frames_imageio(video_path, scenes, output_dir):
    reader = imageio.get_reader(video_path)
    for i, scene in enumerate(scenes):
        start_frame, end_frame = scene
        
        # Convert FrameTimecode to integer frame numbers
        start_frame_num = int(start_frame)
        end_frame_num = int(end_frame)
        
        # Calculate the middle frame of the scene
        middle_frame = (start_frame_num + end_frame_num) // 2
        
        # Set the reader to the middle frame and extract it
        reader.set_image_index(middle_frame)
        frame = reader.get_next_data()
        
        # Save the frame as an image with frame number in the filename
        output_path = os.path.join(output_dir, f'scene_{i + 1}_frame_{middle_frame}.jpg')
        imageio.imwrite(output_path, frame)
        print(f"Extracted and saved middle frame of scene {i + 1} as {output_path}", flush=True)


### 4) Image Processing Function

In [5]:
async def encode_image(image_path):
    async with aiofiles.open(image_path, "rb") as image_file:
        content = await image_file.read()
        return base64.b64encode(content).decode('utf-8')

def get_color_category(color):
    r, g, b = [x / 255.0 for x in color]
    h, l, s = colorsys.rgb_to_hls(r, g, b)

    primary_hues = {
        "red": (0.0, 0.1),  
        "yellow": (0.1, 0.18),
        "green": (0.25, 0.4),
        "blue": (0.55, 0.75),
    }

    for color_name, hue_range in primary_hues.items():
        if hue_range[0] <= h <= hue_range[1]:
            return color_name

    if (l >= 0.9 and s <= 0.1):
        return "white"
    if (l <= 0.1 and s <= 0.1):
        return "black"

    return "non-primary"

def analyze_image_colors(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    data = np.array(image)

    unique_colors, counts = np.unique(data.reshape(-1, data.shape[2]), axis=0, return_counts=True)
    total_pixels = int(counts.sum())

    color_counts = {
        "Red": 0,
        "Yellow": 0,
        "Green": 0,
        "Blue": 0,
        "White": 0,
        "Black": 0,
        "Non-primary": 0
    }

    for color, count in zip(unique_colors, counts):
        category = get_color_category(tuple(color))
        color_counts[category.capitalize()] += int(count)

    color_percentages = {color: (count / total_pixels) * 100 for color, count in color_counts.items()}
    primary_total = color_counts["Red"] + color_counts["Yellow"] + color_counts["Blue"]
    color_dominance = "Primary colors" if primary_total > color_counts["Non-primary"] else "Non-primary colors"

    return {
        "Color Analysis": {
            "Colors Found": {
                color: {
                    "Pixel Count": count,
                    "Percentage": f"{color_percentages[color]:.2f}%"
                } for color, count in color_counts.items()
            },
            "Dominance": color_dominance
        }
    }


### 5) OpenAI API Interaction

In [6]:
async def send_image_to_openai(image_path, base64_image, retries=3):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """
                        Analyze the following image and provide a detailed description in the format of JSON only. Ensure the output is strictly in JSON format without any additional text or code block formatting. The JSON should include the following standardized labels:

                        1. **Image Analysis**: The root dictionary containing all analysis data.
                        
                        2. **Suitability**:
                            - "Nudity": Boolean indicating the presence of nudity.
                            - "Obscene Gestures": Boolean indicating the presence of obscene gestures.
                            - "Alcohol": Boolean indicating the presence of alcohol.
                            - "Drugs": Boolean indicating the presence of drugs.
                            - "Addictions": Boolean indicating the presence of addictions.

                        3. **Objects**:
                            - "Total Objects Identified": Integer representing the total number of objects identified.
                            - "Average Features Per Object": Float representing the average number of features per object.
                            - "Objects Details": Dictionary containing details of each object, where each object is labeled as "Object_1", "Object_2", etc., with the following structure:
                                - "Name": The name of the object - as simplest and descriptive mossible.
                                - "Portion Boolean": 0-1 output indicating if the object is a portion of a larger object (1) or a complete object (0). For example, a leg is a portion of a human. However, if the object is just cropped but clearly identifiable as a complete object, it should be considered a complete object.
                                - "Color": The color of the object.
                                - "Features": List of features of the object.
                                - "Total Features": Integer representing the number of features for the object.

                        4. **Place**:
                            - "Name": The name of the place - as simplest and descriptive mossible.
                            - "Certainty Boolean": 0-1 output indicating if the place is clearly identifiable (1) or not (0).
                            - "Fantasy/Adventurous Place": Boolean (0-1) indicating whether the place is classified as a fantasy/adventurous place or not.
                            - "Explanation": Detailed explanation of why the place is classified as fantasy/adventurous or not. Fantasy places are those that do not exist in reality, and adventurous places are defined as those involving clear statements of traveling to space or another country.

                        5. **Characters**:
                            - "Total Characters Identified": Integer representing the total number of characters identified.
                            - "Average Features Per Character": Float representing the average number of features per character.
                            - "Character Details": Dictionary containing details of each character, where each character is labeled as "Character_1", "Character_2", etc., with the following structure:
                                - "Name": The name of the character - as simplest and descriptive mossible.
                                - "Portion Boolean": 0-1 output indicating if the character is a portion of a larger character (1) or a complete character (0). For example, a leg is a portion of a human. However, if the character is just cropped but clearly identifiable as a complete character, it should be considered a complete character.
                                - "Human or Non-Human": 0-1 output indicating if the character appears human (1) or non-human (0). Anthropomorphized characters or any other combination not fully human are considered non-human.
                                - "Physical Features": List of physical features of the character.
                                - "Explanation": Explanation for why the character is classified as human or non-human, and why these physical features are inferred.
                                - "Age": Expected age range of the character (a single number).
                            **Note**: If the "character" consists of only a part of a body (such as a hand, leg, or face without enough distinguishing features to identify it as a complete character), do not count it as a "character."

                        Ensure that the structure of the JSON output strictly adheres to these standardized labels.
                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 750
    }

    for attempt in range(retries):
        try:
            async with aiohttp.ClientSession() as session:
                async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                    # Log the status code and full response for debugging
                    status = response.status
                    response_text = await response.text()
                    
                    # print(f"Response Status Code: {status}")
                    # print(f"Response Content: {response_text}")

                    if status == 429:
                        print("Rate limit exceeded, retrying...")
                        await asyncio.sleep(2 ** attempt)
                        continue
                    elif status == 200:
                        content = await response.json()
                        
                        # Log the full JSON content
                        # print(f"Full JSON Response for {image_path}: {content}")
                        
                        if 'choices' in content:
                            message_content = content['choices'][0].get('message', {}).get('content', '').strip()
                            try:
                                return json.loads(message_content)
                            except json.JSONDecodeError as e:
                                print(f"Error decoding JSON from OpenAI response for {image_path}: {e}")
                                # print(f"OpenAI Response Content: {message_content}")
                                return None
                        else:
                            print(f"Unexpected response format from OpenAI API for {image_path}.")
                            return None
                    else:
                        print(f"Request failed with status code {status} for {image_path}.")
                        # print(f"Response Content: {response_text}")
                        return None
        except aiohttp.ClientError as e:
            print(f"Request failed due to a client error: {e}")
            await asyncio.sleep(2 ** attempt)
        except Exception as e:
            print(f"Unexpected error occurred: {e}")
            await asyncio.sleep(2 ** attempt)
    return None


### 6) Scene Processing Functions

In [7]:
async def process_scenes_output(output_dir, json_output_dir):
    os.makedirs(json_output_dir, exist_ok=True)
    scenes = sorted([f for f in os.listdir(output_dir) if f.endswith('.jpg')], key=extract_scene_number)
    total_scenes = len(scenes)
    with tqdm(total=total_scenes, desc="Processing Scenes", unit="scene") as pbar:
        tasks = [process_single_scene(i, scene, output_dir, json_output_dir, pbar) for i, scene in enumerate(scenes)]
        await asyncio.gather(*tasks)


async def process_single_scene(i, scene, output_dir, json_output_dir, pbar):
    async with semaphore:  # Limit concurrent execution
        scene_path = os.path.join(output_dir, scene)

        # Encode image in base64
        base64_image = await encode_image(scene_path)

        # Perform color analysis
        color_analysis_result = analyze_image_colors(scene_path)

        # Send image to OpenAI for further analysis
        openai_response = await send_image_to_openai(scene_path, base64_image)

        # Check if openai_response is valid (not None or empty)
        if not openai_response:
            print(f"Skipping {scene} due to invalid OpenAI response.")
            pbar.update(1)
            return

        # Combine both results, and include the reference to the image file
        final_output = {
            "Image File": scene,
            "Image Analysis": {
                **color_analysis_result["Color Analysis"],
                **openai_response.get("Image Analysis", {})
            }
        }

        # The filename already includes the scene number and frame number
        output_filename = os.path.splitext(scene)[0] + '_analysis.json'
        output_path = os.path.join(json_output_dir, output_filename)

        try:
            async with aiofiles.open(output_path, 'w') as json_file:
                await json_file.write(json.dumps(final_output, indent=4))
                print(f"Saved analysis for {scene} as {output_filename}")
        except Exception as e:
            print(f"Failed to save analysis for {scene}: {e}")

        pbar.update(1)


def extract_scene_number(filename):
    match = re.search(r'\d+', filename)
    return int(match.group()) if match else -1

def extract_frame_number(filename):
    match = re.search(r'_frame_(\d+)', filename)
    return int(match.group(1)) if match else -1



### 7) Run whole analysis of each json output

Image Path Construction: get_image_path generates the correct path to the image file based on the JSON filename.

Entity Extraction:extract_entities_from_json pulls characters, objects, and places from the JSON data.

Image-to-Image Comparison:perform_image_to_image_comparison compares partial objects with full objects using the OpenAI API.

Entity Comparison:compare_entities handles both name-based and image-based comparisons to decide whether two entities should be consolidated.

Consolidation:Entities across frames are consolidated into a single summary file that tracks where each entity was found.

Main Execution:The script runs through all JSON files, processes the entities, and saves the consolidated results to a summary JSON file.

Key Features of This Implementation:
Text-Based Comparison: The code first attempts to merge entities based on exact name matches. If no match is found, it uses the OpenAI API to determine if two entities with different names should be merged.

Image-to-Image Comparison: If one of the entities is flagged as a portion, or if names don't match but the entities might still be the same, the code performs an image-to-image comparison using the OpenAI API.

Efficient Processing: The code processes each frame sequentially and logs all merges into merged_entities_log, ensuring you have a record of what entities were merged, including their original names and frames.

No Overwritten Functionality: The original image analysis functionality is preserved and integrated smoothly with the text-based comparisons.

Number of Unique Characters, Objects, and Places: This can be done by counting the keys in the consolidated_data dictionary.
Average Characters per Frame: This can be calculated by summing up all instances of characters found across frames and dividing by the total number of frames where characters appear.
Average Features per Character/Object: Calculate this by summing the features of all characters/objects and dividing by the total number of characters/objects.
Overall Color Analysis: Aggregate the color data from all JSON files.
Filter Compliance: Check for any instances where the filters (e.g., nudity, drugs) are not compliant and log the frame numbers.

In [16]:
import os
import json
import aiohttp
import asyncio
import nest_asyncio
import openai

# Apply nest_asyncio to handle the running event loop
nest_asyncio.apply()

# Initialize OpenAI API
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API key is not set.")
openai.api_key = api_key

#Initial consolidation of entities
def initial_consolidation(json_files):
    consolidated_data = {"characters": {}, "objects": {}, "places": {}}
    merge_tracking = {"characters": {}, "objects": {}, "places": {}}

    for json_file in json_files:
        with open(json_file, 'r') as f:
            json_data = json.load(f)
            consolidate_from_json(json_data, consolidated_data, merge_tracking, os.path.basename(json_file))

    return consolidated_data


# Function to save entities to JSON
def save_entities_to_json(entities, path):
    with open(path, 'w') as f:
        json.dump(entities, f, indent=4)

# Function to consolidate entities from a JSON file
# Function to consolidate entities from a JSON file
def consolidate_from_json(json_data, consolidated_data, merge_tracking, json_file_name):
    frame_number = json_file_name.split('_')[3]  # Extract frame number

    if "Image Analysis" in json_data:
        characters = json_data["Image Analysis"].get("Characters", {}).get("Character Details", {})
        for key, details in characters.items():
            name = details.get("Name")
            if name:
                if name not in consolidated_data["characters"]:
                    consolidated_data["characters"][name] = details
                    consolidated_data["characters"][name]["merged_from"] = []
                    merge_tracking["characters"][name] = {"merged_from": []}
                if frame_number not in consolidated_data["characters"][name]["merged_from"]:
                    consolidated_data["characters"][name]["merged_from"].append(frame_number)
                    merge_tracking["characters"][name]["merged_from"].append(frame_number)

        objects = json_data["Image Analysis"].get("Objects", {}).get("Objects Details", {})
        for key, details in objects.items():
            name = details.get("Name")
            if name:
                if name not in consolidated_data["objects"]:
                    consolidated_data["objects"][name] = details
                    consolidated_data["objects"][name]["merged_from"] = []
                    merge_tracking["objects"][name] = {"merged_from": []}
                if frame_number not in consolidated_data["objects"][name]["merged_from"]:
                    consolidated_data["objects"][name]["merged_from"].append(frame_number)
                    merge_tracking["objects"][name]["merged_from"].append(frame_number)

        place = json_data["Image Analysis"].get("Place", {})
        place_name = place.get("Name")
        if place_name:
            if place_name not in consolidated_data["places"]:
                consolidated_data["places"][place_name] = place
                consolidated_data["places"][place_name]["merged_from"] = []
                merge_tracking["places"][place_name] = {"merged_from": []}
            if frame_number not in consolidated_data["places"][place_name]["merged_from"]:
                consolidated_data["places"][place_name]["merged_from"].append(frame_number)
                merge_tracking["places"][place_name]["merged_from"].append(frame_number)


# Function to cluster entities using OpenAI API and name the clusters
async def cluster_entities(api_key, entities):
    # Generate lists for characters, objects, and places from entities
    character_list = ', '.join(entities['characters'].keys())
    object_list = ', '.join(entities['objects'].keys())
    place_list = ', '.join(entities['places'].keys())

    if not character_list and not object_list and not place_list:
        return "No entities available to cluster."

    # Adjusted OpenAI prompt to return structured output (dictionaries)
    prompt = f"""
    You are tasked with clustering and naming entities from a TV show. Below are lists of characters, objects, and places extracted from different scenes. These lists sometimes contain multiple labels for the same entity.

    **Instructions:**

    1. Group the characters, objects, and places that refer to the same entity and suggest a single **final name** for each group (be smart what could be the same individual/object in the show one and what couldn't).
    2. Return the result as a dictionary where each cluster (key) contains the entities (values) that belong to that cluster.
    3. Use this format:

    {{
      "Characters Clusters": {{
        "Final Name 1": ["Character 1", "Character 2", ...],
        "Final Name 2": ["Character 3", "Character 4", ...]
      }},
      "Objects Clusters": {{
        "Final Name 1": ["Object 1", "Object 2", ...],
        "Final Name 2": ["Object 3", "Object 4", ...]
      }},
      "Places Clusters": {{
        "Final Name 1": ["Place 1", "Place 2", ...],
        "Final Name 2": ["Place 3", "Place 4", ...]
      }}
    }}

    **Characters:**
    {character_list}

    **Objects:**
    {object_list}

    **Places:**
    {place_list}
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    payload = {
        "model": "gpt-4",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 2000
    }

    async with aiohttp.ClientSession() as session:
        try:
            async with session.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload) as response:
                response_json = await response.json()
                clusters = response_json.get('choices', [{}])[0].get('message', {}).get('content', '')
                return json.loads(clusters)  # Convert the response to JSON
        except Exception as e:
            print(f"Error accessing API response: {e}")
            return {}

# Function to merge clusters with final_final_entities.json, keeping names and frame numbers
def merge_clusters_with_entities(final_entities, clusters):
    merged_entities = {"characters": {}, "objects": {}, "places": {}}

    # Process characters
    character_clusters = clusters.get("Characters Clusters", {})
    for final_name, cluster_items in character_clusters.items():
        merged_entities["characters"][final_name] = {"merged_from": [], "merged_names": []}
        for item in cluster_items:
            if item in final_entities["characters"]:
                entity_data = final_entities["characters"][item]
                merged_entities["characters"][final_name] = {
                    **entity_data,
                    "Name": final_name,
                    "merged_from": list(set(merged_entities["characters"][final_name]["merged_from"] + entity_data["merged_from"])),
                    "merged_names": list(set(merged_entities["characters"][final_name]["merged_names"] + [item]))
                }

    # Process objects
    object_clusters = clusters.get("Objects Clusters", {})
    for final_name, cluster_items in object_clusters.items():
        merged_entities["objects"][final_name] = {"merged_from": [], "merged_names": []}
        for item in cluster_items:
            if item in final_entities["objects"]:
                entity_data = final_entities["objects"][item]
                merged_entities["objects"][final_name] = {
                    **entity_data,
                    "Name": final_name,
                    "merged_from": list(set(merged_entities["objects"][final_name]["merged_from"] + entity_data["merged_from"])),
                    "merged_names": list(set(merged_entities["objects"][final_name]["merged_names"] + [item]))
                }

    # Process places
    place_clusters = clusters.get("Places Clusters", {})
    for final_name, cluster_items in place_clusters.items():
        merged_entities["places"][final_name] = {"merged_from": [], "merged_names": []}
        for item in cluster_items:
            if item in final_entities["places"]:
                entity_data = final_entities["places"][item]
                merged_entities["places"][final_name] = {
                    **entity_data,
                    "Name": final_name,
                    "merged_from": list(set(merged_entities["places"][final_name]["merged_from"] + entity_data["merged_from"])),
                    "merged_names": list(set(merged_entities["places"][final_name]["merged_names"] + [item]))
                }

    return merged_entities
# Main async function for clustering and merging entities
async def cluster_and_merge_entities(api_key, json_output_dir, video_folder_path):
    json_files = [os.path.join(json_output_dir, f) for f in os.listdir(json_output_dir) if f.endswith('.json')]

    if not json_files:
        print("No JSON files found for consolidation.")
        return

    # Consolidate entities from all JSON files
    final_entities = initial_consolidation(json_files)  # Ensure only json_files is passed here
    print(final_entities)

    # Cluster entities using OpenAI API
    clusters = await cluster_entities(api_key, final_entities)

    if not clusters:
        print("No clusters returned from OpenAI API.")
        return

    # Merge clusters into final final JSON
    merged_final_entities = merge_clusters_with_entities(final_entities, clusters)

    # Save the merged entities to JSON
    merged_final_entities_path = os.path.join(video_folder_path, 'final_summary.json')
    save_entities_to_json(merged_final_entities, merged_final_entities_path)
    print(f"Merged entities saved to {merged_final_entities_path}")

    # Update the final JSON with summary statistics
    update_final_json_with_summary(merged_final_entities_path)



In [17]:
## other stats

In [20]:
# Function to calculate summary statistics
def generate_summary_statistics(final_entities):
    stats = {
        "unique_characters_count": len(final_entities["characters"]),
        "unique_objects_count": len(final_entities["objects"]),
        "unique_places_count": len(final_entities["places"]),
        "average_physical_features_per_character": 0,
        "average_features_per_object": 0,
        "average_frames_per_character": 0
    }

    # Calculate average physical features for characters
    characters = final_entities.get("characters", {})
    if len(characters) > 0:
        total_features_characters = sum(len(character.get("Physical Features", [])) for character in characters.values())
        stats["average_physical_features_per_character"] = total_features_characters / len(characters)

    # Calculate average features for objects
    objects = final_entities.get("objects", {})
    if len(objects) > 0:
        total_features_objects = sum(len(obj.get("Features", [])) for obj in objects.values())
        stats["average_features_per_object"] = total_features_objects / len(objects)

    # Calculate average frames per character
    if len(characters) > 0:
        total_frames_characters = sum(len(character.get("merged_from", [])) for character in characters.values())
        stats["average_frames_per_character"] = total_frames_characters / len(characters)

    return stats

# # Example usage
# final_json_path = '/path/to/your/final_summary.json'
# update_final_json_with_summary(final_json_path)
# print(f"Updated final JSON with summary statistics saved to {final_json_path}")


### 8) Main Function Execution

In [21]:
import os
import time
import asyncio
from tqdm import tqdm

# Ensure the function is called correctly and used in processing
def process_videos_in_directory(directory_path, output_base_dir):
    video_files = [f for f in os.listdir(directory_path) if f.endswith(('.mp4', '.avi', '.mkv'))]

    if not video_files:
        print("No video files found in the directory.", flush=True)
        return

    with tqdm(total=len(video_files), desc="Processing Videos", unit="video") as pbar:
        for i, video_file in enumerate(video_files):
            start_time = time.time()

            try:
                video_path = os.path.join(directory_path, video_file)
                video_name = os.path.splitext(video_file)[0]
                video_size = os.path.getsize(video_path)

                video_output_dir = os.path.join(output_base_dir, video_name)
                scenes_output_dir = os.path.join(video_output_dir, 'scenes_output')
                json_output_dir = os.path.join(video_output_dir, 'json_output')

                os.makedirs(scenes_output_dir, exist_ok=True)
                os.makedirs(json_output_dir, exist_ok=True)

                print(f"Processing video {i + 1}/{len(video_files)}: {video_file}", flush=True)

                scenes = analyze_video(video_path)
                extract_frames_imageio(video_path, scenes, scenes_output_dir)
                asyncio.run(process_scenes_output(scenes_output_dir, json_output_dir))  # Run async scene processing

                # Ensure final_entities.json is created in the correct location
                final_json_path = os.path.join(json_output_dir, 'final_entities.json')
                asyncio.run(cluster_and_merge_entities(api_key, json_output_dir, video_output_dir))

                end_time = time.time()
                processing_time = end_time - start_time

                # Calculate video length (in seconds)
                video_length = get_video_length(video_path)

                # Calculate scenes per minute
                scenes_per_minute = len(scenes) / (video_length / 60)

                print(f"Finished processing video: {video_file}", flush=True)
                pbar.update(1)
            except Exception as e:
                print(f"Error processing video {video_file}: {e}", flush=True)


# Ensure the main script has appropriate paths
video_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/02_Video_DB'
output_base_directory = '/Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB'

process_videos_in_directory(video_directory, output_base_directory)
print("FINISHED PROCESSING ALL VIDEOS.", flush=True)


Processing Videos:   0%|          | 0/1 [00:00<?, ?video/s]

Processing video 1/1: Bananas_in_pyjamas copy.mp4


ERROR:pyscenedetect:VideoManager is deprecated and will be removed.
INFO:pyscenedetect:Loaded 1 video, framerate: 24.969 FPS, resolution: 640 x 360
INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...
INFO:root:Detected 3 scenes:
INFO:root:Scene 1: Start 00:00:00.000 / Frame 0, End 00:00:08.370 / Frame 209
INFO:root:Scene 2: Start 00:00:08.370 / Frame 209, End 00:00:19.344 / Frame 483
INFO:root:Scene 3: Start 00:00:19.344 / Frame 483, End 00:00:22.147 / Frame 553


Extracted and saved middle frame of scene 1 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas copy/scenes_output/scene_1_frame_104.jpg
Extracted and saved middle frame of scene 2 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas copy/scenes_output/scene_2_frame_346.jpg
Extracted and saved middle frame of scene 3 as /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas copy/scenes_output/scene_3_frame_518.jpg


Saved analysis for scene_1_frame_104.jpg as scene_1_frame_104_analysis.json


...

Processing Scenes: 100%|██████████| 3/3 [00:17<00:00,  5.84s/scene]

Saved analysis for scene_2_frame_346.jpg as scene_2_frame_346_analysis.json
{'characters': {'Banana Character 1': {'Name': 'Banana Character 1', 'Portion Boolean': 0, 'Human or Non-Human': 0, 'Physical Features': ['Yellow skin', 'Big eyes', 'Smiling mouth'], 'Explanation': 'The character is anthropomorphized, resembling a banana with human-like features, thus classified as non-human.', 'Age': 5, 'merged_from': ['346', '518']}, 'Banana Character 2': {'Name': 'Banana Character 2', 'Portion Boolean': 0, 'Human or Non-Human': 0, 'Physical Features': ['Yellow skin', 'Big eyes', 'Smiling mouth'], 'Explanation': 'The character is anthropomorphized, resembling a banana with human-like features, thus classified as non-human.', 'Age': 5, 'merged_from': ['346', '518']}}, 'objects': {'House': {'Name': 'House', 'Portion Boolean': 0, 'Color': 'Blue and White', 'Features': ['Roof', 'Walls', 'Door', 'Windows'], 'Total Features': 4, 'merged_from': ['104']}, 'Trees': {'Name': 'Trees', 'Portion Boolean':

Merged entities saved to /Users/santiagowon/Dropbox/Santiago/01. Maestria/Tesis/11_Project_Analysed_DB/Bananas_in_pyjamas copy/final_summary.json
Finished processing video: Bananas_in_pyjamas copy.mp4


Processing Videos: 100%|██████████| 1/1 [00:25<00:00, 25.52s/video]

FINISHED PROCESSING ALL VIDEOS.
